Esta nota utiliza métodos vistos en [1.5.Integracion_numerica](https://github.com/ITAM-DS/analisis-numerico-computo-cientifico/blob/master/temas/I.computo_cientifico/1.5.Integracion_numerica.ipynb)

**Notas para contenedor de docker:**

Comando de docker para ejecución de la nota de forma local:

nota: cambiar `<ruta a mi directorio>` por la ruta de directorio que se desea mapear a `/datos` dentro del contenedor de docker.

```
docker run --rm -v <ruta a mi directorio>:/datos --name jupyterlab_numerical -p 8888:8888 -p 8786:8786 -p 8787:8787 -d palmoreck/jupyterlab_numerical:1.1.0
```

Detener el contenedor de docker:

```
docker stop jupyterlab_local
```


Documentación de la imagen de docker `palmoreck/jupyterlab_numerical:1.1.0` en [liga](https://github.com/palmoreck/dockerfiles/tree/master/jupyterlab/numerical).

---

Documentación de [cython](https://cython.org/): 

* [Basic Tutorial](https://cython.readthedocs.io/en/latest/src/tutorial/cython_tutorial.html)

* [Source Files and Compilation](https://cython.readthedocs.io/en/latest/src/userguide/source_files_and_compilation.html)

* [Compiling with a Jupyter Notebook](https://cython.readthedocs.io/en/latest/src/userguide/source_files_and_compilation.html#compiling-with-a-jupyter-notebook)

Instalamos cython:

In [1]:
%pip install -q --user cython

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
You should consider upgrading via the 'pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import IPython
app = IPython.Application.instance()
app.kernel.do_shutdown(True)

In [1]:
import math
import time
from scipy.integrate import quad

Función a compilar en un archivo `.pyx`:

In [2]:
%%file Rcf_cython.pyx
def Rcf(f,a,b,n): #Rcf: rectángulo compuesto para f
    """
    Compute numerical approximation using rectangle or mid-point method in 
    an interval.
    Nodes are generated via formula: x_i = a+(i+1/2)h_hat for i=0,1,...,n and h_hat=(b-a)/n
    Args:
        f (lambda expression): lambda expression of integrand
        a (int): left point of interval
        b (int): right point of interval
        n (int): number of subintervals
    Returns:
        Rcf (float) 
    """
    h_hat=(b-a)/n
    nodes=[a+(i+1/2)*h_hat for i in range(0,n)]
    sum_res=0
    for node in nodes:
        sum_res=sum_res+f(node)
    return h_hat*sum_res


Writing Rcf_cython.pyx


Archivo `setup.py` que contiene las instrucciones para el build:

In [3]:
%%file setup.py
from distutils.core import setup
from distutils.extension import Extension
from Cython.Distutils import build_ext
setup(cmdclass = {'build_ext': build_ext},ext_modules = [Extension("Rcf_compiled", ["Rcf_cython.pyx"])])

Writing setup.py


Compilar:

In [4]:
%%bash
python3 setup.py build_ext --inplace #inplace para compilar el módulo en el directorio
                                     #actual

running build_ext
cythoning Rcf_cython.pyx to Rcf_cython.c
building 'Rcf_compiled' extension
creating build
creating build/temp.linux-x86_64-3.6
x86_64-linux-gnu-gcc -pthread -DNDEBUG -g -fwrapv -O2 -Wall -g -fstack-protector-strong -Wformat -Werror=format-security -Wdate-time -D_FORTIFY_SOURCE=2 -fPIC -I/usr/include/python3.6m -c Rcf_cython.c -o build/temp.linux-x86_64-3.6/Rcf_cython.o
x86_64-linux-gnu-gcc -pthread -shared -Wl,-O1 -Wl,-Bsymbolic-functions -Wl,-Bsymbolic-functions -Wl,-z,relro -Wl,-Bsymbolic-functions -Wl,-z,relro -g -fstack-protector-strong -Wformat -Werror=format-security -Wdate-time -D_FORTIFY_SOURCE=2 build/temp.linux-x86_64-3.6/Rcf_cython.o -o /datos/MNO_desde_2018/ramas_repo/mno-master/temas/I.computo_cientifico/Rcf_compiled.cpython-36m-x86_64-linux-gnu.so


/home/miuser/.local/lib/python3.6/site-packages/Cython/Compiler/Main.py:369: FutureWarning: Cython directive 'language_level' not set, using 2 for now (Py2). This will change in a later release! File: /datos/MNO_desde_2018/ramas_repo/mno-master/temas/I.computo_cientifico/Rcf_cython.pyx
  tree = Parsing.p_module(s, pxd, full_module_name)


Importar módulo compilado y ejecutarlo:

In [5]:
f=lambda x: math.exp(-x**2) #using math library

In [6]:
import Rcf_compiled


In [7]:
n=10**6
start_time = time.time()
aprox=Rcf_compiled.Rcf(f,0,1,n)
end_time = time.time()


In [8]:
aprox

0.746824448872663

In [9]:
secs = end_time-start_time
print("Rcf tomó",secs,"segundos" )

Rcf tomó 0.3168480396270752 segundos


Revisar error relativo:

In [10]:
def err_relativo(aprox, obj):
    return math.fabs(aprox-obj)/math.fabs(obj) #obsérvese el uso de la librería math

In [11]:
obj, err = quad(f, 0, 1)
err_relativo(aprox,obj)

4.2320570799153987e-07

**<- checar este valor de error relativo**

In [12]:
%timeit -n 5 -r 10 Rcf_compiled.Rcf(f,0,1,n)

318 ms ± 16 ms per loop (mean ± std. dev. of 10 runs, 5 loops each)


In [13]:
%%file Rcf_cython2.pyx
def Rcf(f,a,b,n): #Rcf: rectángulo compuesto para f
    """
    Compute numerical approximation using rectangle or mid-point method in 
    an interval.
    Nodes are generated via formula: x_i = a+(i+1/2)h_hat for i=0,1,...,n and h_hat=(b-a)/n
    Args:
        f (lambda expression): lambda expression of integrand
        a (int): left point of interval
        b (int): right point of interval
        n (int): number of subintervals
    Returns:
        Rcf (float) 
    """
    h_hat=(b-a)/n
    nodes=[a+(i+1/2)*h_hat for i in range(0,n)]
    sum_res=0
    for node in nodes:
        sum_res=sum_res+f(node)
    return h_hat*sum_res


Writing Rcf_cython2.pyx


In [14]:
%%file setup2.py
from distutils.core import setup
from Cython.Build import cythonize

setup(ext_modules = cythonize("Rcf_cython2.pyx", compiler_directives={'language_level' : 3}))


Writing setup2.py


In [15]:
%%bash
python3 setup2.py build_ext --inplace

Compiling Rcf_cython2.pyx because it changed.
[1/1] Cythonizing Rcf_cython2.pyx
running build_ext
building 'Rcf_cython2' extension
x86_64-linux-gnu-gcc -pthread -DNDEBUG -g -fwrapv -O2 -Wall -g -fstack-protector-strong -Wformat -Werror=format-security -Wdate-time -D_FORTIFY_SOURCE=2 -fPIC -I/usr/include/python3.6m -c Rcf_cython2.c -o build/temp.linux-x86_64-3.6/Rcf_cython2.o
x86_64-linux-gnu-gcc -pthread -shared -Wl,-O1 -Wl,-Bsymbolic-functions -Wl,-Bsymbolic-functions -Wl,-z,relro -Wl,-Bsymbolic-functions -Wl,-z,relro -g -fstack-protector-strong -Wformat -Werror=format-security -Wdate-time -D_FORTIFY_SOURCE=2 build/temp.linux-x86_64-3.6/Rcf_cython2.o -o /datos/MNO_desde_2018/ramas_repo/mno-master/temas/I.computo_cientifico/Rcf_cython2.cpython-36m-x86_64-linux-gnu.so


In [16]:
import Rcf_cython2

In [17]:
n=10**6
start_time = time.time()
aprox=Rcf_cython2.Rcf(f,0,1,n)
end_time = time.time()


In [18]:
aprox

0.7468241328124773

In [19]:
secs = end_time-start_time
print("Rcf tomó",secs,"segundos" )

Rcf tomó 0.3310391902923584 segundos


Revisar error relativo:

In [20]:
obj, err = quad(f, 0, 1)
err_relativo(aprox,obj)

6.71939731300312e-14

Con magic `%Cython`

In [21]:
%load_ext Cython

In [22]:
%%cython
def Rcf_magic(f,a,b,n): #Rcf: rectángulo compuesto para f
    """
    Compute numerical approximation using rectangle or mid-point method in 
    an interval.
    Nodes are generated via formula: x_i = a+(i+1/2)h_hat for i=0,1,...,n and h_hat=(b-a)/n
    Args:
        f (lambda expression): lambda expression of integrand
        a (int): left point of interval
        b (int): right point of interval
        n (int): number of subintervals
    Returns:
        Rcf (float) 
    """
    h_hat=(b-a)/n
    nodes=[a+(i+1/2)*h_hat for i in range(0,n)]
    sum_res=0
    for node in nodes:
        sum_res=sum_res+f(node)
    return h_hat*sum_res


In [23]:
n=10**6
start_time = time.time()
aprox=Rcf_magic(f,0,1,n)
end_time = time.time()

In [24]:
secs = end_time-start_time
print("Rcf tomó",secs,"segundos" )

Rcf tomó 0.3288137912750244 segundos


In [25]:
obj, err = quad(f, 0, 1)
err_relativo(aprox,obj)

6.71939731300312e-14

In [26]:
%timeit -n 5 -r 10 Rcf_magic(f,0,1,n)

328 ms ± 24.7 ms per loop (mean ± std. dev. of 10 runs, 5 loops each)


Analizar bloque de código con cython:

In [28]:
%%bash
$HOME/.local/bin/cython -a Rcf_cython.pyx

/home/miuser/.local/lib/python3.6/site-packages/Cython/Compiler/Main.py:369: FutureWarning: Cython directive 'language_level' not set, using 2 for now (Py2). This will change in a later release! File: /datos/MNO_desde_2018/ramas_repo/mno-master/temas/I.computo_cientifico/Rcf_cython.pyx
  tree = Parsing.p_module(s, pxd, full_module_name)


Ver archivo creado: `Rcf_cython.html`

Com magic y flag:

In [29]:
%%cython?

Docstring:
::

  %cython [-a] [-+] [-3] [-2] [-f] [-c COMPILE_ARGS]
              [--link-args LINK_ARGS] [-l LIB] [-n NAME] [-L dir] [-I INCLUDE]
              [-S SRC] [--pgo] [--verbose]

Compile and import everything from a Cython code cell.

The contents of the cell are written to a `.pyx` file in the
directory `IPYTHONDIR/cython` using a filename with the hash of the
code. This file is then cythonized and compiled. The resulting module
is imported and all of its symbols are injected into the user's
namespace. The usage is similar to that of `%%cython_pyximport` but
you don't have to pass a module name::

    %%cython
    def f(x):
        return 2.0*x

To compile OpenMP codes, pass the required  `--compile-args`
and `--link-args`.  For example with gcc::

    %%cython --compile-args=-fopenmp --link-args=-fopenmp
    ...

To enable profile guided optimisation, pass the ``--pgo`` option.
Note that the cell itself needs to take care of establishing a suitable
profile when executed. 

In [30]:
%%cython -a
def Rcf2_magic(f,a,b,n): #Rcf: rectángulo compuesto para f
    """
    Compute numerical approximation using rectangle or mid-point method in 
    an interval.
    Nodes are generated via formula: x_i = a+(i+1/2)h_hat for i=0,1,...,n and h_hat=(b-a)/n
    Args:
        f (lambda expression): lambda expression of integrand
        a (int): left point of interval
        b (int): right point of interval
        n (int): number of subintervals
    Returns:
        Rcf (float) 
    """
    h_hat=(b-a)/n
    sum_res=0
    for i in range(0,n):
        x=a+(i+1/2.0)*h_hat
        sum_res+=f(x)
    return h_hat*sum_res


In [31]:
n=10**6
start_time = time.time()
aprox=Rcf2_magic(f,0,1,n)
end_time = time.time()

In [32]:
secs = end_time-start_time
print("Rcf tomó",secs,"segundos" )

Rcf tomó 0.366257905960083 segundos


In [33]:
obj, err = quad(f, 0, 1)
err_relativo(aprox,obj)

6.71939731300312e-14

In [34]:
%timeit -n 5 -r 10 Rcf2_magic(f,0,1,n)

313 ms ± 24.2 ms per loop (mean ± std. dev. of 10 runs, 5 loops each)


Usando cdef's

In [35]:
%%cython -a
def Rcf3_magic(f,double a,double b,unsigned int n): #Rcf: rectángulo compuesto para f
    """
    Compute numerical approximation using rectangle or mid-point method in 
    an interval.
    Nodes are generated via formula: x_i = a+(i+1/2)h_hat for i=0,1,...,n and h_hat=(b-a)/n
    Args:
        f (lambda expression): lambda expression of integrand
        a (int): left point of interval
        b (int): right point of interval
        n (int): number of subintervals
    Returns:
        Rcf (float) 
    """
    cdef unsigned int i
    cdef double x,sum_res, h_hat
    h_hat=(b-a)/n
    sum_res=0
    for i in range(0,n):
        x=a+(i+1/2.0)*h_hat
        sum_res+=f(x)
    return h_hat*sum_res

In [36]:
n=10**6
start_time = time.time()
aprox=Rcf3_magic(f,0,1,n)
end_time = time.time()

In [37]:
secs = end_time-start_time
print("Rcf tomó",secs,"segundos" )

Rcf tomó 0.2413010597229004 segundos


In [38]:
obj, err = quad(f, 0, 1)
err_relativo(aprox,obj)

6.71939731300312e-14

In [39]:
%timeit -n 5 -r 10 Rcf3_magic(f,0,1,n)

238 ms ± 16.4 ms per loop (mean ± std. dev. of 10 runs, 5 loops each)


Mejor tiempo:

In [40]:
%%cython -a
import math
def Rcf4_magic(double a,double b,unsigned int n): #Rcf: rectángulo compuesto para f
    """
    Compute numerical approximation using rectangle or mid-point method in 
    an interval.
    Nodes are generated via formula: x_i = a+(i+1/2)h_hat for i=0,1,...,n and h_hat=(b-a)/n
    Args:
        f (lambda expression): lambda expression of integrand
        a (int): left point of interval
        b (int): right point of interval
        n (int): number of subintervals
    Returns:
        Rcf (float) 
    """
    cdef unsigned int i
    cdef double x,sum_res, h_hat
    h_hat=(b-a)/n
    sum_res=0
    for i in range(0,n):
        x=a+(i+1/2.0)*h_hat
        sum_res+=math.exp(-x**2)
    return h_hat*sum_res

In [41]:
n=10**6
start_time = time.time()
aprox=Rcf4_magic(0,1,n)
end_time = time.time()

In [42]:
secs = end_time-start_time
print("Rcf tomó",secs,"segundos" )

Rcf tomó 0.0923771858215332 segundos


In [43]:
err_relativo(aprox,obj)

6.71939731300312e-14

In [44]:
%timeit -n 5 -r 10 Rcf4_magic(0,1,n)

81 ms ± 2.78 ms per loop (mean ± std. dev. of 10 runs, 5 loops each)


**Ejercicios**

1. Realiza el análisis con las herramientas revisadas en esta nota para las reglas del trapecio y de Simpson de  la nota [1.5.Integracion_numerica](https://github.com/ITAM-DS/analisis-numerico-computo-cientifico/blob/master/temas/I.computo_cientifico/1.5.Integracion_numerica.ipynb).


**Referencias**

1. M. Gorelick, I. Ozsvald, High Performance Python, O'Reilly Media, 2014.